In [1]:
import librosa

In [3]:
mels = librosa.filters.mel(sr=22050, n_fft=2048, n_mels=256)

In [4]:
mels.shape

(256, 1025)

In [5]:
mels[0]

array([0.        , 0.06423059, 0.02601504, ..., 0.        , 0.        ,
       0.        ], dtype=float32)